# 라이브러리 Import
- 공통적으로 사용하는 라이브러리를 불러옵니다.

In [1]:
import os
from dotenv import load_dotenv

## 기존 VertexAI SDK 사용
- google.oauth2의 service_account를 활용 
- GCP의 Credential 파일을 통해 별도 로그인 없이 VertexAI 사용

In [16]:
from google.oauth2 import service_account
import vertexai
from vertexai.generative_models import (
    GenerativeModel, 
    GenerationConfig,
    Tool,
    grounding,
)

- VERTEXAI_PROJECT_ID: Vertex AI 사용 권한이 있는 GCP 프로젝트의 ID
- VERTEXAI_CREDENTIALS_PATH: Vertex AI에 접근 권한이 있는 서비스 계정의 “JSON 키 파일” 경로
- **별도 Client를 받아 올 필요없이, init 한 번으로 호출 끝**

In [4]:
load_dotenv()

project_id = os.getenv("VERTEXAI_PROJECT_ID")
credential_path = os.getenv("VERTEXAI_CREDENTIALS_PATH")
credentials = service_account.Credentials.from_service_account_file(credential_path)

vertexai.init(
    project=project_id, 
    credentials=credentials
)

### Vertex AI 답변 생성 함수

In [ ]:
def vertexai_generate(
    prompt,
    model_name="gemini-1.5-flash",
    generation_config=GenerationConfig(),
    tools=None
):
    model = GenerativeModel(
        model_name=model_name,
        tools=tools,
        generation_config=generation_config,
    )

    response = model.generate_content(prompt)
    return response.text

- 기본 값(Default)으로 답변

In [ ]:
prompt = "2025년 한국 대통령 누구야?"

result = vertexai_generate(prompt)
print("result: ", result, flush=True)

result:  아직 2025년 한국 대통령이 누가 될지는 알 수 없습니다.  2022년 대선 이후로는  대한민국 대통령 선거가 5년마다 치러지기 때문에 다음 대선은 2027년에 있을 예정입니다.  따라서 2025년에는 현직 대통령이 계속해서 직무를 수행할 것입니다.



- 최신 모델(gemini 2.5 버전) 사용

In [ ]:
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = vertexai_generate(prompt, model_name)
print("result: ", result, flush=True)

result:  2025년 한국 대통령은 현재와 동일하게 **윤석열** 대통령입니다.

윤석열 대통령은 2022년 5월 10일에 취임했으며, 대한민국 대통령의 임기는 5년 단임이므로 2027년 5월까지 재임하게 됩니다.


- Config 변경

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.95,
    top_k=20,
    candidate_count=1,
    seed=5,
)
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = vertexai_generate(prompt, model_name, generation_config)
print("result: ", result, flush=True)

result:  현재 대한민국 대통령은 **윤석열 대통령**입니다.

대한민국 대통령의 임기는 5년 단임이며, 윤석열 대통령은 2022년 5월에 취임했습니다.

따라서 **2025년에도 윤석열 대통령이 재임 중일 것입니다.** 다음 대통령 선거는 2027년에 치러질 예정입니다.


- Tools 사용(Google 웹 검색)
    - Vertex AI에서 2.0 이상 모델은 사용 가능하지만, **웹 검색은 1.5 버전까지만 지원**

In [ ]:
google_search_retrieval = grounding.GoogleSearchRetrieval()
tools = [Tool.from_google_search_retrieval(google_search_retrieval)]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.95,
    top_k=20,
    candidate_count=1,
    seed=5,
)
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = vertexai_generate(prompt, model_name, generation_config, tools)
print("result: ", result, flush=True)

InvalidArgument: 400 Unable to submit request because google_search_retrieval is not supported; please use google_search field instead. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

- 1.5 버전으로 모델 변경 후 재실행

In [ ]:
google_search_retrieval = grounding.GoogleSearchRetrieval()
tools = [Tool.from_google_search_retrieval(google_search_retrieval)]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.95,
    top_k=20,
    candidate_count=1,
    seed=5,
)
model_name = "gemini-1.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = vertexai_generate(prompt, model_name, generation_config, tools)
print("result: ", result, flush=True)

result:  2025년 대한민국 대통령은 이재명입니다.  2025년 6월 3일에 치러진 제21대 대통령 선거에서 당선되었습니다.  윤석열 전 대통령의 탄핵으로 인한 조기 대선이었습니다.



## 신규 GenAI SDK 사용
- GenAI SDK는 GEMINI API를 사용하지만, 기존 VertexAI API를 필요에 따라 끌어 올 수 있다.
- google.oauth2의 service_account 필요 없음
- **Gemini API Key를 통해 Client 호출**

In [47]:
from google import genai
from google.genai.types import (
    Content,
    Part,
    GoogleSearch,
    GenerateContentConfig,
    Tool
)

1) VertexAI API 사용 시
    - VERTEXAI_USE: Vertex AI 사용 유무
    - VERTEXAI_PROJECT_ID: Vertex AI 사용 권한이 있는 GCP 프로젝트의 ID
    - VERTEXAI_LOCATION: Vertex AI에 접근 권한이 있는 GCP 프로젝트의 LOCATION

In [45]:
load_dotenv()

# 반드시 해줘야 함
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("VERTEXAI_CREDENTIALS_PATH")

client = genai.Client(
    vertexai=True,
    project="gai-llm-poc",
    location="us-central1",
)

2) 순수 GEMINI API 사용 시
    - GENAI_API_KEY: GEMINI API 활성화 시 발급된 API Key

In [50]:
load_dotenv()

api_key = os.getenv("GENAI_API_KEY")

client = genai.Client(
    vertexai=False,
    api_key=api_key
)

### Gen AI 답변 생성 함수

In [36]:
def genai_generate(
    client,
    prompt,
    model_name="gemini-1.5-flash",
    generation_config=GenerateContentConfig(),
):
    contents = []
    contents.append(
        Content(
            role="user",
            parts=[
                Part.from_text(text=prompt)
            ]
        ),
    )
    response = client.models.generate_content(
        model=model_name,
        contents=contents,
        config=generation_config,
    )
    return response.text

- 기본 값(Default)으로 답변

In [38]:
prompt = "2025년 한국 대통령 누구야?"

result = genai_generate(client, prompt)
print("result: ", result, flush=True)

result:  2025년 한국 대통령은 아직 결정되지 않았습니다.  2022년 대통령 선거에서 윤석열 후보가 당선되었고, 그의 임기는 2022년 5월 10일부터 2027년 5월 9일까지 입니다. 따라서 2025년에도 윤석열 대통령이 한국의 대통령일 것입니다.



- 최신 모델(gemini 2.5 버전) 사용

In [41]:
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = genai_generate(client, prompt, model_name)
print("result: ", result, flush=True)

result:  2025년 한국 대통령은 **윤석열 대통령**입니다.

윤석열 대통령은 2022년 5월에 취임했으며, 대한민국 대통령의 임기는 5년 단임이기 때문에 2027년 5월까지가 그의 임기입니다. 따라서 2025년에도 윤석열 대통령이 재임 중입니다.


- Config 변경

In [44]:
generation_config = GenerateContentConfig(
    temperature=0.1,
    top_p=0.95,
    top_k=20,
    candidate_count=1,
    seed=5,
)
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = genai_generate(client, prompt, model_name, generation_config)
print("result: ", result, flush=True)

result:  2025년에도 **윤석열** 대통령입니다.

대한민국 대통령의 임기는 5년 단임제이며, 윤석열 대통령은 2022년 5월 10일에 취임했으므로, 임기는 2027년 5월 9일까지입니다. 따라서 2025년에는 윤석열 대통령이 재임 중입니다.


- Tools 사용(Google 웹 검색)
    - **2.0 이상 모델 또한 웹 검색 사용 가능**
    - Tools가 GenerateContentConfig 안에 포함되는 구조

In [52]:
google_search = GoogleSearch()
tools = [Tool(google_search=google_search)]
generation_config = GenerateContentConfig(
    temperature=0.1,
    top_p=0.95,
    top_k=20,
    candidate_count=1,
    seed=5,
    tools=tools  # 안에 포함됨
)
model_name = "gemini-2.5-flash"
prompt = "2025년 한국 대통령 누구야?"

result = genai_generate(client, prompt, model_name, generation_config)
print("result: ", result, flush=True)

result:  2025년 대한민국의 대통령은 이재명입니다. 그는 2025년 6월 3일에 치러진 제21대 대통령 선거에서 당선되었으며, 임기는 2025년 6월 4일부터 2030년 6월 3일까지입니다. 이 선거는 윤석열 전 대통령의 탄핵으로 인해 조기에 실시되었습니다.
